In [288]:
# 환경구축 : requirements.txt + pandas install
import numpy as np
import pandas as pd
import pyproj
import geopy
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings(action='ignore')

import os
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

path = '/opt/ml/'

In [3]:
pharmacy = pd.read_csv('pharmacy.csv',encoding='cp949')

In [6]:
pharmacy['영업상태명'].unique()

array(['폐업', '영업/정상', '취소/말소/만료/정지/중지', '휴업', '제외/삭제/전출'], dtype=object)

## 영업상태코드와 동일, 1:영업, 2:휴업, 3:폐업, 4:취소/말소/정지, 5:삭제/제외

## 영업중인 약국만 사용

In [13]:
pd.options.display.max_columns = 100

In [26]:
pharmacy[pharmacy['영업상태코드']==1].shape

(5500, 27)

In [27]:
pharmacy[pharmacy['영업상태명']=='영업/정상'].shape

(5500, 27)

In [28]:
pharmacy[pharmacy['상세영업상태명']=='영업중'].shape

(5499, 27)

In [35]:
# pharmacy[(pharmacy['영업상태명']=='영업/정상') & (pharmacy['상세영업상태명']!='영업중')]

# 가공 시작

In [276]:
p1 = pharmacy[pharmacy['영업상태코드']==1]

In [277]:
tmp = p1[['관리번호','사업장명','도로명주소','전화번호','약국영업면적','개방자치단체코드','인허가일자','영업상태코드']]

In [278]:
# pharmacy[pharmacy['사업장명']=='김상돈약국']

In [279]:
tmp.drop(3306,axis=0,inplace=True) # 정보없는 데이터 삭제

In [280]:
tmp[:2]

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,개방자치단체코드,인허가일자,영업상태코드
2,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645(면목동),02-493-8856,NaN,3060000,20221017,1
3,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41 1층 120 120-1호 (암사동 암사동양덱스빌),02-428-2588,NaN,3240000,20221017,1


In [281]:
def address_preprocessing(x):
    s = str(x).split()
    for i in range(len(s)):
        if s[i][-1]=='로' or s[i][-1]=='길':
            return ' '.join(s[:i+2])
    else:
        return 0

In [282]:
address_preprocessing('서울특별시 중랑구 동일로 645(면목동)')

'서울특별시 중랑구 동일로 645(면목동)'

In [283]:
tmp['도로명주소'] = tmp['도로명주소'].apply(lambda x: address_preprocessing(x))
tmp['도로명주소'] = tmp['도로명주소'].apply(lambda x: x.split('(')[0])

In [305]:
tmp.shape

(5499, 9)

In [106]:
# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [293]:
tmp['위경도'] = tmp['도로명주소'].apply(lambda x: geocoding(x))

In [313]:
tmp[:2]

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,개방자치단체코드,인허가일자,영업상태코드,위경도
2,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,3060000,20221017,1,"[37.5986537, 127.0798699]"
3,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,3240000,20221017,1,"[37.54671, 127.15022]"


In [314]:
for index, row in tmp.iterrows():
    tmp.loc[index,'위도'] = row['위경도'][0]
    tmp.loc[index,'경도'] = row['위경도'][1]

In [315]:
tmp[:2]

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,개방자치단체코드,인허가일자,영업상태코드,위경도,위도,경도
2,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,3060000,20221017,1,"[37.5986537, 127.0798699]",37.598654,127.07987
3,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,3240000,20221017,1,"[37.54671, 127.15022]",37.546710,127.15022


In [317]:
final_p = tmp[['관리번호','사업장명','도로명주소','전화번호','약국영업면적','인허가일자','영업상태코드','위도','경도']]

In [318]:
final_p[:2]

,관리번호,사업장명,도로명주소,전화번호,약국영업면적,인허가일자,영업상태코드,위도,경도
2,PHMD120223060034084000006,연세파란약국,서울특별시 중랑구 동일로 645,02-493-8856,NaN,20221017,1,37.598654,127.07987
3,PHMD120223240033084000019,더조은약국,서울특별시 강동구 상암로 41,02-428-2588,NaN,20221017,1,37.546710,127.15022


In [320]:
final_p.to_csv('pharmacy_data.csv',index=False)

# ---------------------------

In [308]:
tmp[tmp['위경도'].apply(lambda x: x==[0,0])].shape

(53, 9)

In [304]:
geocoding('서울특별시 관악구 남부순환로 2082')

[37.4821966, 126.9474016]

In [309]:
tmp['위경도'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[0, 0]                               53
[37.5739132, 127.0188445]            51
[37.6242653, 127.0734496]            49
[37.6410017, 127.0289955]            45
[37.5526401, 127.1441574]            39
                                     ..
[37.6063975, 127.036435]              1
[37.556239, 127.0350264]              1
[37.6092679, 127.0095134]             1
[37.49254025, 127.14478070000001]     1
[37.4729269, 127.0407513]             1
Name: 위경도, Length: 2301, dtype: int64

In [312]:
# tmp[tmp['위경도'].apply(lambda x: x==[37.5739132, 127.0188445])]

In [322]:
from haversine import haversine

In [337]:
haversine((37,127),(30,127))

778.3655616347307

In [341]:
haversine((37,127),(37,127.011749478646))

1.0434046913400234

In [342]:
haversine((37.009,127),(37,127))

1.000755722102017